In [1]:
!pip install -q transformers==4.44.2 soundfile==0.12.1 sentencepiece==0.2.0 numpy==1.26.4 torchaudio==2.4.0


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
import numpy as np
import soundfile as sf
import re
from IPython.display import Audio as IPyAudio, display
from transformers import SpeechT5ForTextToSpeech, SpeechT5Processor, SpeechT5HifiGan

DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Device: {DEVICE}")

model_inf = SpeechT5ForTextToSpeech.from_pretrained("./speecht5_tts_colombian_final").to(DEVICE)
processor_inf = SpeechT5Processor.from_pretrained("./speecht5_tts_colombian_final")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan").to(DEVICE)
model_inf.eval()
spk_emb_avg = torch.tensor(np.load("./speaker_embedding.npy"), dtype=torch.float32).unsqueeze(0).to(DEVICE)

/Users/juandiego/Documents/study/keepcoding/Final Proyect/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: mps


In [5]:
def preparar_texto(texto):
    texto = texto.replace("¿", "").replace("¡", "")
    texto = texto.replace(",", "").replace(".", "")
    texto = texto.replace(":", "").replace(";", "")
    
    return re.sub(r'\s+', ' ', texto).strip()


def dividir_en_frases(texto, max_tokens=50):

    PATRONES_CORTE = [
        r',\s+',
        r'\s+(pero|sino|aunque|porque|cuando|donde|como|mientras|después|antes|entonces|ya que)\s+',
        r'\s+(y|e|o|u)\s+(?=\w{4,})',
    ]
    
    n_tokens = processor_inf(text=texto, return_tensors="pt")["input_ids"].shape[1]
    if n_tokens <= max_tokens:
        return [texto]

    for patron in PATRONES_CORTE:
        segmentos = re.split(f'({patron})', texto, flags=re.IGNORECASE)
        partes_candidatas = []
        chunk = ""

        for seg in segmentos:
            chunk += seg
            tok = processor_inf(text=chunk.strip(), return_tensors="pt")["input_ids"].shape[1]

            if tok >= max_tokens:
                anterior = chunk[:-(len(seg))].strip()
                if anterior:
                    partes_candidatas.append(anterior)
                chunk = seg.strip()

        if chunk.strip():
            partes_candidatas.append(chunk.strip())
        if all(processor_inf(text=p, return_tensors="pt")["input_ids"].shape[1] <= max_tokens for p in partes_candidatas) and len(partes_candidatas) > 1:
            return partes_candidatas
        
    palabras = texto.split()
    partes, chunk_actual = [], []
    for palabra in palabras:
        chunk_actual.append(palabra)

        if processor_inf(text=" ".join(chunk_actual), return_tensors="pt")["input_ids"].shape[1] >= max_tokens:
            partes.append(" ".join(chunk_actual))
            chunk_actual = []

    if chunk_actual:
        partes.append(" ".join(chunk_actual))
        
    return partes


def recortar_silencio(audio, sr=16000, margen_ms=150):
    umbral = max(audio.abs().max().item() * 0.05, 0.001)
    margen = int(sr * margen_ms / 1000)
    indices = (audio.abs() > umbral).nonzero(as_tuple=True)[0]
    
    if len(indices) == 0:
        return audio

    return audio[max(0, indices[0].item() - margen):min(len(audio), indices[-1].item() + margen)]


def generar_chunk(texto, threshold=0.5, minlenratio=0.1):
    inputs = processor_inf(text=texto.strip() + " .", return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        speech = model_inf.generate_speech(
            inputs["input_ids"],
            speaker_embeddings=spk_emb_avg,
            vocoder=vocoder,
            threshold=threshold,
            minlenratio=minlenratio,
            maxlenratio=20.0,
        )

    return speech.cpu()


def intentar_generar(parte, amp_minima=0.05, profundidad=0):
    for params in [{"threshold": 0.5, "minlenratio": 0.1}, {"threshold": 0.4, "minlenratio": 0.2}, {"threshold": 0.6, "minlenratio": 0.0}]:
        speech = generar_chunk(parte, params["threshold"], params["minlenratio"])
        if speech.abs().max().item() >= amp_minima:
            return [recortar_silencio(speech)]

    palabras = parte.strip().split()
    if len(palabras) <= 2 or profundidad >= 3:
        return []

    mitad = len(palabras) // 2
    return (
        intentar_generar(" ".join(palabras[:mitad]), amp_minima, profundidad + 1)
        + [torch.zeros(1600)]
        + intentar_generar(" ".join(palabras[mitad:]), amp_minima, profundidad + 1)
    )


def generar_audio(texto, nombre="output.wav", max_tokens=50):
    
    texto_limpio = preparar_texto(texto)
    partes = dividir_en_frases(texto_limpio, max_tokens)
    partes_filtradas = []
    
    for parte in partes:
        n_tok = processor_inf(text=parte, return_tensors="pt")["input_ids"].shape[1]
        if n_tok < 15 and partes_filtradas:
            partes_filtradas[-1] += " " + parte
        else:
            partes_filtradas.append(parte)
    partes = partes_filtradas

    audios = []
    for j, parte in enumerate(partes):
        audios += intentar_generar(parte)
        if j < len(partes) - 1:
            audios.append(torch.zeros(3200))

    if not audios:
        print("Error audio")
        return

    audio_final = torch.cat(audios)
    if audio_final.abs().max().item() > 0.001:
        sf.write(nombre, audio_final.numpy(), samplerate=16000)
        display(IPyAudio(nombre))
    else:
        print("Audio silencioso")

In [ ]:
generar_audio("Keep Coding es el mejor bootcamp de inteligencia artificial", "result.wav")